# TF2

Save models for TF2.X using the tf2 env (py 3.10 tf 2.8).

Re-create the architecture (trying to load models from 1.14 results in bad marshal data that stems from some issue with the lambda funciton). Replacing keras backend K with tf.math to make it easier to load.

Save in SavedModel format.

In [9]:
import glob
import tensorflow as tf

In [10]:
tf.__version__

'2.8.0'

In [11]:
! python --version

Python 3.10.6


In [12]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

In [13]:
import keras

def bpnet_seq(seqlen=2000, filters=256, ndil=9):
    """
    Classic BPNet architecture with sequence-only input. Predicts profile
    logits and log-counts.

    Inputs and outputs both have length seqlen
    """

    inp = keras.Input((seqlen,4))

    x = keras.layers.Conv1D(filters, 
                            kernel_size=25, 
                            padding='same', 
                            activation='relu')(inp)

    for i in range(1, ndil+1):
        conv_x = keras.layers.Conv1D(filters, 
                                     kernel_size=3, 
                                     padding='same', 
                                     activation='relu', 
                                     dilation_rate=2**i)(x)
        x = keras.layers.add([conv_x, x])
    bottleneck = x     

    prof = keras.layers.Conv1D(1, 25, padding='same')(bottleneck)    
    prof = keras.layers.Flatten(name="logits")(prof)

    ct = keras.layers.GlobalAvgPool1D()(bottleneck)
    ct = keras.layers.Dense(1, name="logcounts")(ct)

    return keras.Model(inputs=inp, outputs=[prof,ct])


def chrombpnet(seqlen=2000, filters=256, ndil=9):
    """
    ChromBPNet architecture with sequence + bias counts + bias profile logits
    as inputs. Passes sequence through a bpnet_seq module. The counts output
    are logsumexp-ed with bias counts input, and the profile output is simply 
    added to the bias logits input.
    """

    # Inputs: sequence + (predicted) bias logits + (predicted) bias logcounts
    inp = keras.Input((seqlen,4))
    inp_bias_logits = keras.Input(shape=(seqlen,))
    inp_bias_logcounts = keras.Input(shape=(1,))

    # sequence is processed by bpnet_seq model
    seq_model = bpnet_seq(seqlen=seqlen,
                          filters=filters,
                          ndil=ndil)
    unbias_logits, unbias_logcounts = seq_model(inp)

    # PROFILE: predicted logits are simply a sum of unbias logits with 
    # (predicted) bias logits
    out_logits = keras.layers.Add(name="logits_w_bias")([unbias_logits, inp_bias_logits]) # <== add in bias logits
    
    # COUNTS: final count is log(exp(unbias_logcounts) + exp(inp_bias_logcounts)))
    concat_cts = keras.layers.concatenate([unbias_logcounts, inp_bias_logcounts],
                                          axis=-1)
    out_logcounts = keras.layers.Lambda(
                        lambda x: tf.math.reduce_logsumexp(x, axis=-1, keepdims=True),
                        name="logcounts_w_bias")(concat_cts)

    return keras.Model(inputs=[inp, inp_bias_logits, inp_bias_logcounts], 
                       outputs=[out_logits, out_logcounts])

In [14]:
bias_model = bpnet_seq()
chrombpnet_model = chrombpnet()

In [15]:
for fold in range(10):
    for m in glob.glob("./intermediate/fold{}/*".format(fold)):
        fname = m.split("/")[-1]
        
        if "bias" in fname:
            bias_model.load_weights(m)
            bias_model.save("./out/tf2.8/fold{}/{}".format(fold, fname[:-3])) # strip .h5

        else:
            assert("chrombpnet" in fname)
            chrombpnet_model.load_weights(m)
            chrombpnet_model.save("./out/tf2.8/fold{}/{}".format(fold, fname[:-3]))
            
        print(m, fname)
#         break

2023-08-14 15:12:28.019889: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ./out/tf2.8/fold0/c1_fibroblast.chrombpnet/assets
./intermediate/fold0/c1_fibroblast.chrombpnet.h5 c1_fibroblast.chrombpnet.h5
INFO:tensorflow:Assets written to: ./out/tf2.8/fold0/c3_fibroblast_like.chrombpnet/assets
./intermediate/fold0/c3_fibroblast_like.chrombpnet.h5 c3_fibroblast_like.chrombpnet.h5
INFO:tensorflow:Assets written to: ./out/tf2.8/fold0/c10_partially_reprogrammed.bias/assets
./intermediate/fold0/c10_partially_reprogrammed.bias.h5 c10_partially_reprogrammed.bias.h5
INFO:tensorflow:Assets written to: ./out/tf2.8/fold0/c14_pre_ipsc.chrombpnet/assets
./intermediate/fold0/c14_pre_ipsc.chrombpnet.h5 c14_pre_ipsc.chrombpnet.h5
INFO:tensorflow:Assets written to: ./out/tf2.8/fold0/c1_fibroblast.bias/assets
./intermediate/fold0/c1_fibroblast.bias.h5 c1_fibroblast.bias.h5
INFO:tensorflow:Assets written to: ./out/tf2.8/fold0/c13_pre_ipsc.chrombpnet/assets
./intermediate/fold0/c13_pre_ipsc.chrombpnet.h5 c13_pre_ipsc.chrombpnet.h5
INFO:tensorflow: